# Title & Objective

# Data Loading

# Data Overview & Variable Types

# EDA (4 variables + business interpretation)

# Feature Engineering (≥4 engineered features)

# Train / Validation / Test strategy

# Model 1 (Algorithm + tuning + feature importance)

# Model 2 (Algorithm + tuning + feature importance)

# Model comparison

# Threshold optimization (business cost/benefit)

# Final test evaluation

# Conclusion
